<a href="https://colab.research.google.com/github/hansuho113/AlgorithmBook/blob/master/Colab_Notebook/Suho/drug_top3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive  
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display as dp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
drug_res = pd.read_table("/content/drive/MyDrive/SW21Project/TCGA_Drug_Response_07272017.txt")
drug_res

,Cancer Type,bcr_patient_barcode,drug_name,measure_of_response,regimen_indication,days_to_drug_therapy_start,days_to_drug_therapy_end,therapy_types
0,ACC,TCGA-PK-A5HB,Sorafenib,Clinical Progressive Disease,NaN,968.0,1029.0,Targeted Molecular therapy
1,ACC,TCGA-OR-A5K2,Adriamycin,NaN,NaN,118.0,298.0,Chemotherapy
2,ACC,TCGA-OR-A5K2,Cisplatin,NaN,NaN,118.0,298.0,Chemotherapy
3,ACC,TCGA-OR-A5K2,carboplatin,NaN,NaN,118.0,299.0,Chemotherapy
4,ACC,TCGA-OR-A5K2,etoposide,NaN,NaN,118.0,298.0,Chemotherapy
...,...,...,...,...,...,...,...,...
12566,UCS,TCGA-N5-A4RU,Cisplatin,Complete Response,NaN,48.0,113.0,Chemotherapy
12567,UVM,TCGA-V4-A9F3,fotemustine,NaN,NaN,27.0,175.0,Chemotherapy
12568,UVM,TCGA-V4-A9F1,fotemustine,NaN,NaN,22.0,147.0,Chemotherapy
12569,UVM,TCGA-V4-A9ET,fotemustine,NaN,NaN,21.0,174.0,Chemotherapy


In [4]:
drug_res['Cancer Type'].value_counts()

OV      2516
BRCA    2406
GBM     1505
LGG      695
COAD     593
UCEC     479
STAD     463
LUAD     461
LUSC     357
HNSC     353
BLCA     303
SKCM     265
PAAD     254
READ     231
DLBC     225
SARC     208
CESC     199
TGCT     171
KIRC     168
MESO     159
PRAD     130
UCS       95
ESCA      82
LIHC      66
THCA      56
KIRP      47
KICH      21
ACC       18
THYM      16
CHOL      14
PCPG      11
UVM        4
Name: Cancer Type, dtype: int64

- OV: ovarian cancer: 난소암
- BRCA: breast cancer: 유방암
- GBM: glioblastoma cancer: 교모세포종 (뇌)
- LGG: Low grade glioma: 저등급신경교종 (뇌)
- COAD: Colon adenocarcinoma: 대장암
- UCEC: Uterine Corpus Endometrial Carcinoma: 자궁내막암

In [5]:
drug_coad = drug_res[drug_res['Cancer Type'] == "COAD"]
drug_ov = drug_res[drug_res['Cancer Type'] == 'OV']
drug_brca = drug_res[drug_res['Cancer Type'] == 'BRCA']
drug_ucec = drug_res[drug_res['Cancer Type'] == 'UCEC']

In [12]:
print(f"""
    각 암종별 환자 데이터 수
    대장암: {len(drug_coad.bcr_patient_barcode.unique().tolist())}
    난소암: {len(drug_ov.bcr_patient_barcode.unique().tolist())}
    유방암: {len(drug_brca.bcr_patient_barcode.unique().tolist())}
    자궁내막암: {len(drug_ucec.bcr_patient_barcode.unique().tolist())}
""")


    각 암종별 환자 데이터 수
    대장암: 154
    난소암: 530
    유방암: 780
    자궁내막암: 201



In [13]:
drug_list = drug_res['drug_name'].unique().tolist()
drug_for_coad = drug_coad['drug_name'].unique().tolist()
drug_for_ov = drug_ov['drug_name'].unique().tolist()
drug_for_brca = drug_brca['drug_name'].unique().tolist()
drug_for_ucec = drug_ucec['drug_name'].unique().tolist()

print(f"""
drug_response 데이터셋의 전체 약물: {len(drug_list)}

COAD(대장암) 사용 약물: {len(drug_for_coad)}
OV(난소암) 사용 약물: {len(drug_for_ov)}
BRCA(유방암) 사용 약물: {len(drug_for_brca)}
UCEC(자궁내막암) 사용 약물: {len(drug_for_ucec)}
""")


drug_response 데이터셋의 전체 약물: 1065

COAD(대장암) 사용 약물: 66
OV(난소암) 사용 약물: 172
BRCA(유방암) 사용 약물: 204
UCEC(자궁내막암) 사용 약물: 65



In [15]:
drug_coad.drug_name.unique()

array(['5-FU', 'Oxaliplatin', 'Cetuximab Study drug', 'Leucovorin',
       '5 FU', 'Folinic acid', '5-Fluorouracil', nan, 'XELODA', '5FU',
       'Fluorouracil', 'oxaliplatin', 'Folinic Acid', 'Cetuximab',
       'Bevacizumab', 'Irinotecan', 'Avastin', 'Xeloda', 'Folfox',
       'Capecitabine', 'capecitabine', 'Floxuridine', 'Dexamethasone',
       'Irinotecan HCl', 'Pegfilgrastim', 'Leucovorin Calcium',
       'IRINOtecan HCl', 'Mitomycin', 'Camptosar',
       'Calcium Foliatum, fluorouracilum, oxaliplatinum, dexamethassone',
       'oxaliplatinum+5 fluorouracilum', '5- FU', 'Raltitrexed',
       'leucovorin', 'fluorouracil', 'Erbitux', 'avastin', 'irinotecan',
       'leucovorin calcium', 'bevacizumab', 'regorafenib',
       'Pegfilgrastim (Peg G-CSF)', 'Filgrastim (G-CSF)',
       'Irinotecan HCL', 'C1 FOLFIRI/Zaltrap', 'Irinotecan + Cetuximab',
       'FOLFIRI/Avastin', 'FOLFOX', '5-Fluorouracil + leucovorin',
       'FolFox', 'Oxaliplatin, Folinic acid, Fluorouracil', 'Panitumumab

In [ ]:
drug_ov.drug_name.unique()

array(['Taxotere', 'Cisplatin', 'Topotecan', 'Taxol', 'Doxil',
       'Mitomycin C', 'Carboplatin', 'Tamoxifen', 'Gemcitabine',
       'Navelbine', 'Paciltaxel', 'Paclitaxel', 'Gemzet', 'Etoposide',
       'Cytoxan', 'Liposomal Doxorubicin', 'Carboplatin #1', 'Hexalen',
       'carboplatin', 'Gemzar', 'Cisplatin #2-7', 'Avastin', 'Patupilone',
       'Cediranib Placebo', 'Docetaxel', nan, 'VP 16', 'Armidex',
       'Letrozole', 'Lupron', 'Toptecan', 'Pacliatxel', 'Doxetaxel',
       'Oxaliplatin', 'Abagovomab or Placebo', 'Interferon gamma',
       'Vamydex', 'Cisplatinum', 'Gemcitibine', 'Carboplatinum',
       'Abagovomab vs Placebo', 'Femara', 'Taxoterecin', 'Carbo',
       'Hexalin', 'Cetuximab', 'Doxorubicin HCL', 'Doxorubicin',
       'Bevacizumab or placebo', 'Carbplatin', 'Topetecan',
       'Bevacizumab or Placebo', 'Adriamycin', 'Trabectedin',
       'Sargramostin', 'paclitaxel', 'SNS 595', 'Taxol or Taxotere',
       'Bevacizumab', 'Xyotax', 'topotecan', 'taxol', 'Paclitaxil

In [ ]:
brca_drug = drug_brca.drug_name.unique().tolist()
brca_drug  = [str(drug).upper() for drug in brca_drug]
len(brca_drug)

204

In [ ]:
unique_brca_drug = list(set(brca_drug))
len(unique_brca_drug)

147

In [ ]:
unique_brca_drug

['DOXIL',
 'ARIMIDEX',
 'VINORELBINE',
 'CYCLOPHOSPHAMIDE+METHOTREXATUM+FLUOROURACILLUM',
 'TESETAXEL',
 'FLUOROURACIL',
 'TAMOXIPHENE+ANASTROZOLUM',
 'NEULASTA',
 'ADRIAMYCIN+CYCLOPHOSPHAMIDE',
 'CYCLOPHOSPHAMIDE',
 'CHEMO, NOS',
 'DOXORUBICINUM',
 'AC',
 'CLODRONIC ACID',
 'ADRIAMICIN',
 'LEUPROLIDE',
 'LETROZOL',
 'METOTREKSAT',
 'DOXORUBICINE',
 'NAVELBINE',
 'ADRIAMYCIN, CYTOXAN, AVASTIN',
 'TAMOXIPHENE+LEUPORELINE+GOSERELINE',
 'ADRIAMYCIN+CUCLOPHOSPHAMIDE',
 '5-FLOUROURACIL',
 'DOXORUBICIN LIPOSOME',
 'TAXOTERE/CYTOXAN',
 'CYCLOPHOSPHAMIDUM',
 'TAMOXIPHEN+ANASTRAZOLUM',
 'TAXOL',
 'AROMASIN',
 'METHOTREXATE+5 FLUOROURACIL+CYCLOPHOSPHAMIDE',
 'ADRIMYCIN',
 'LEUPROLIDE ACETATE (LUPRON)',
 'NAN',
 'MITOMYCIN',
 'XELODA',
 'TC',
 'GOSERELIN',
 'LEUPRORELIN',
 'IXABEPILONE',
 'ADRIAMYCIN+CYCLOPHOSPHAMID',
 'TAXANE',
 'FARESTON',
 'TAMOXIFEN CITRATE',
 'TAMOXIFEN (NOVADEX)',
 'AVASTIN',
 '5-FLUOROURACIL',
 'CYCLOPHOSPHANE',
 'ALOXI',
 'LAPATINIB',
 'CISPLATIN',
 'RITUXIMAB',
 'LETROZO